Experiment with OpenCV for finding the pool table edges within an image

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
def ImgShow(images, resolution=120):
    plt.rcParams['figure.dpi'] = resolution
    
    cnt = len(images)
    if cnt > 20:
        # Top-level object is an image, make it an array of 1
        images = [images]
        cnt = len(images)
        
    cols = cnt
    fig, ax = plt.subplots(1, cnt)    
    if 1 == cnt:
        ax = [ax]
    for i in range(0, len(images)):
        dims = len(images[i].shape)
        if 3 == dims:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_BGR2RGB)
        else:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_GRAY2RGB)
            
        ax[i].axis('off')
        ax[i].imshow(imgDisp)

In [ ]:
# Load the test image
imageOriginal = cv.imread("../test/images/long_all.jpg")

In [ ]:
#Find the edges using flood, start with a blur
blur = int(25)
imageBlur = cv.GaussianBlur(imageOriginal, (blur,blur), cv.BORDER_DEFAULT)

# Edge detect using canny
imageCanny = cv.Canny(imageOriginal, 100, 175)

ImgShow([imageBlur, imageCanny],220)

In [ ]:
# Contours
imgGray = cv.cvtColor(imageOriginal, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(imgGray, 127, 255, 0)
contours, b, = cv.findContours(imageCanny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
img = imageOriginal.copy()
cv.drawContours(img, contours, -1, (0,255,0),3)
ImgShow([imageCanny,img])